In [1]:
!which python

/opt/anaconda3/bin/python


In [2]:
print("OK")

OK


In [3]:
%pwd

'/Users/shubh/Desktop/HSBC /FinGuide/research'

In [4]:
import os 
os.chdir("../")

In [5]:
%pwd

'/Users/shubh/Desktop/HSBC /FinGuide'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_files("data")

In [9]:
extracted_data

[Document(metadata={'producer': 'Adobe PDF library 17.00', 'creator': 'Adobe Illustrator 28.7 (Windows)', 'creationdate': '2025-06-11T15:16:44+06:30', 'author': 'Kriti MANTRI', 'moddate': '2025-06-11T15:16:44+05:30', 'title': 'MITC-home-loans_11-6-25', 'source': 'data/mitc-home-loans.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='This MITC covers following loan products: \ni.  Home Loan\nii.  Smart Home Loan **\niii.  Loan Against Property (LAP) \niv.  Smart Loan Against Property ** \nv.  Smart Home Drop-line Overdraft (Smart Home DLOD)\nvi.  Smart Loan Against Property Drop-line Overdraft (Smart LAP DLOD)\n1. Fees and Charges:\n• A list of our fees and charges applicable on our Home Loan products are set out in the tariﬀ section of this document and \nare also updated on our website (www.hsbc.co.in). They are also available upon request at any of our branches across India.\n• Please be advised that indirect taxes any other amount liable to tax under prevailing I

In [10]:
len(extracted_data)

13

In [11]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [12]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [13]:
minimal_docs

[Document(metadata={'source': 'data/mitc-home-loans.pdf'}, page_content='This MITC covers following loan products: \ni.  Home Loan\nii.  Smart Home Loan **\niii.  Loan Against Property (LAP) \niv.  Smart Loan Against Property ** \nv.  Smart Home Drop-line Overdraft (Smart Home DLOD)\nvi.  Smart Loan Against Property Drop-line Overdraft (Smart LAP DLOD)\n1. Fees and Charges:\n• A list of our fees and charges applicable on our Home Loan products are set out in the tariﬀ section of this document and \nare also updated on our website (www.hsbc.co.in). They are also available upon request at any of our branches across India.\n• Please be advised that indirect taxes any other amount liable to tax under prevailing Indirect tax Laws would be recovered \non all our fees and charges.\n• For Additional information, you can visit any of our branches in India or call HSBC Phone Banking service.\n \nTariﬀ  sheet   \nHome Loan/Smart Home Loan/LAP/Smart LAP/Smart Home DLOD/Smart LAP DLOD \nProcessing 

In [14]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [15]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 140


In [16]:
texts_chunk

[Document(metadata={'source': 'data/mitc-home-loans.pdf'}, page_content='This MITC covers following loan products: \ni.  Home Loan\nii.  Smart Home Loan **\niii.  Loan Against Property (LAP) \niv.  Smart Loan Against Property ** \nv.  Smart Home Drop-line Overdraft (Smart Home DLOD)\nvi.  Smart Loan Against Property Drop-line Overdraft (Smart LAP DLOD)\n1. Fees and Charges:\n• A list of our fees and charges applicable on our Home Loan products are set out in the tariﬀ section of this document and'),
 Document(metadata={'source': 'data/mitc-home-loans.pdf'}, page_content='are also updated on our website (www.hsbc.co.in). They are also available upon request at any of our branches across India.\n• Please be advised that indirect taxes any other amount liable to tax under prevailing Indirect tax Laws would be recovered \non all our fees and charges.\n• For Additional information, you can visit any of our branches in India or call HSBC Phone Banking service.\n \nTariﬀ  sheet   \nHome Loan/

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-mpnet-base-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

embedding = download_embeddings()

/var/folders/h4/_1g5fgpx203d5hpzfqcygfkm0000gn/T/ipykernel_18803/1965188103.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
vector = embedding.embed_query("Hello world")
vector

[0.0262497179210186,
 0.01339566521346569,
 -0.00453313160687685,
 -0.021791428327560425,
 0.0545518659055233,
 -0.004966484382748604,
 0.006655567325651646,
 0.03062623180449009,
 -0.005762856919318438,
 -0.004562049172818661,
 -0.0033132920507341623,
 -0.048496320843696594,
 -0.011364065110683441,
 0.03507746756076813,
 0.09309467673301697,
 -0.086687371134758,
 0.05108654871582985,
 0.009886188432574272,
 -0.06356936693191528,
 -0.008550235070288181,
 0.007054397836327553,
 -0.003862350480630994,
 0.024744274094700813,
 0.042884934693574905,
 0.03509414568543434,
 -0.029848305508494377,
 0.010252607055008411,
 0.02234489843249321,
 0.02088998816907406,
 0.009492207318544388,
 -0.033044345676898956,
 -0.012284120544791222,
 0.05352889746427536,
 0.025429150089621544,
 2.022177341132192e-06,
 -0.03419102355837822,
 0.009610000997781754,
 -0.016484517604112625,
 0.005609503481537104,
 -0.004250052385032177,
 -0.022801216691732407,
 0.04035478085279465,
 0.00305197830311954,
 0.03137262

In [20]:
print( "Vector length:", len(vector))

Vector length: 768


In [21]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [22]:
PINECONE_API_KEY = os.getenv("PINECODE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")



os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [23]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [24]:
pc

In [25]:
from pinecone import ServerlessSpec 

index_name = "finguidenew"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=768,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [26]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [27]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs = retriever.invoke("I have a Smart Home DLOD account. What are the three conditions that could cause my account to be classified as a Non-Performing Asset (NPA), even if I have surplus funds in it?")
retrieved_docs

[Document(id='c3017750-1404-472a-ba1a-22530dd71cfe', metadata={'source': 'data/mitc-home-loans.pdf'}, page_content='*90 days refer to rolling 90 days wherein last 90 days debits / credits are checked daily and NPA classiﬁcation is done accordingly.\n Borrower(s) have understood the above example and understand that if any of the conditions illustrated above is not met then the  \n Smart Home / Smart LAP DLOD will immediately get classiﬁed as non-performing asset (NPA), despite keeping surplus funds in  \nSmart Home / Smart LAP DLOD.'),
 Document(id='3db57ada-40fb-4fbc-9e1d-45ded7268484', metadata={'source': 'data/mitc-home-loans.pdf'}, page_content='*90 days refer to rolling 90 days wherein last 90 days debits / credits are checked daily and NPA classiﬁcation is done accordingly.\n Borrower(s) have understood the above example and understand that if any of the conditions illustrated above is not met then the  \n Smart Home / Smart LAP DLOD will immediately get classiﬁed as non-performi

In [30]:
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [31]:
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.getenv("GEMINI_API_KEY"))

In [32]:
prompt_template = PromptTemplate.from_template("""
You are a helpful assistant for HSBC Clarity Engine where you give answers to queries related to HSBC Questions. Use the context below to answer the question.
**Core Instructions:**
**Handle "I Don't Know" Scenarios:** If the answer is not found in the provided context, you must state: "I do not have the information on that specific topic. I can assist with questions regarding home loans, card services, and account statements."
**Safety and Compliance Guardrails:**
* **Refuse Inappropriate Queries:** You must politely refuse to answer any questions that involve vulgarity, hate speech, or offensive language. A suitable response is: "I cannot assist with that request."
* **Protect Customer Privacy:** You must NEVER ask for, repeat, or acknowledge any personally identification or if a customer asks for someones else personal details just politely say "I cannot provide you with these details"
Context:
{context}

Question: {input}

Answer:
""")

In [33]:
question_answer_chain = create_stuff_documents_chain(chat_model, prompt_template)

In [34]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
query = "What are the three conditions that could cause my Smart Home DLOD account to be NPA despite surplus funds?"
response = rag_chain.invoke({"input": query})

In [36]:
print(response["answer"])

I do not have the information on that specific topic. I can assist with questions regarding home loans, card services, and account statements.


In [37]:
query = "Are there any charges if I want to prepay my floating-rate home loan?"
response = rag_chain.invoke({"input": query})

In [38]:
print(response["answer"])

No, there are no prepayment charges for floating-rate home loans.


In [39]:
query = "How much does it cost to switch from a floating rate to a fixed rate loan?"
response = rag_chain.invoke({"input": query})

In [40]:
print(response["answer"])

The cost to switch from a floating rate to a fixed rate loan is ₹5,000.  Stamping/franking costs will be an additional expense borne by the borrower.


In [41]:
query = "How long do I have to wait after my loan is disbursed before I can switch my interest rate type for the first time?"
response = rag_chain.invoke({"input": query})

In [42]:
print(response["answer"])

The provided text doesn't specify a waiting period after disbursement before the first interest rate switch.  However, it does state that a minimum 12-month gap is required between *any two* switches.


In [43]:
query = "How do you calculate switching of intrest rates?"
response = rag_chain.invoke({"input": query})

In [44]:
print(response["answer"])

The switching charges depend on the type of rate change:

* **Floating rate to Fixed rate:** A flat fee of ₹5,000 is charged.
* **Fixed rate to Floating rate:** A fee of ₹5,000 plus 5% of the outstanding principal balance is charged.

Stamping/franking costs will be borne by the borrower.  You are allowed up to three switches during the loan tenure, with the first switch permitted 12 months after the disbursement date.


### Mitigating Compliance Risk:

This question is a classic compliance trap. Reserve Bank of India (RBI) and FEMA guidelines have very strict rules about the end-use of loans for Non-Resident Indians (NRIs). A human agent, trying to be helpful but perhaps not recalling the specific clause under pressure, might give a vague or incorrect answer, potentially exposing the bank to regulatory penalties for improper advice.

In [45]:
query = "I am an NRI. Can I take out a home loan and use the funds to invest in the Indian stock market?"
response = rag_chain.invoke({"input": query})

In [46]:
print(response["answer"])

Based on the provided text, while NRIs can take out home loans, the loan amount cannot be remitted outside India and should not be credited into NRE/FCNR (B) accounts.  The text does not explicitly forbid using home loan funds for investment in the Indian stock market, but it strongly implies that such use is against the terms.  Therefore, I would advise against it.  To be certain, I recommend contacting HSBC directly to clarify the permissible uses of home loan funds for NRIs.


In [47]:
query = "Why would my account be marked as an NPA if I have more than the EMI amount saved in it?"
response = rag_chain.invoke({"input": query})

In [48]:
print(response["answer"])

I do not have the information on that specific topic. I can assist with questions regarding home loans, card services, and account statements.


In [49]:
query = "What a bullshit it is"
response = rag_chain.invoke({"input": query})

In [50]:
print(response["answer"])

I cannot assist with that request.
